# Sensitivity Analysis (SA) with VARS and G-VARS:
## Correlated/Uncorrelated and Uniformly/Non-uniformly Distributed Inputs

Notebook developed by Saman Razavi, Cordell Blanchard, and Kasra Keshavarz


### For the VARS method, please cite:

Razavi, S., & Gupta, H. V. (2016). A new framework for comprehensive, robust, and efficient global sensitivity analysis: 1. Theory. Water Resources Research, 52(1), 423-439. https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2015WR017558

Razavi, S., & Gupta, H. V. (2016). A new framework for comprehensive, robust, and efficient global sensitivity analysis: 2. Application. Water Resources Research, 52(1), 440-455. https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2015WR017559

### For the Generalized VARS (G-VARS) method, please cite:

Do, N. C., & Razavi, S. (2020). Correlation effects? A major but often neglected component in sensitivity and uncertainty analysis. Water Resources Research, 56(3), e2019WR025436. https://doi.org/10.1029/2019WR025436

## Exercise 1: From A to Z on the Ishigami test function
### Objectives:
Experiment 1 | First, let's run SA on Ishigami via *VARS* when the inputs are uncorrelated and uniformly distributed - this is the classic setting in the literature and the objective is to set a benchmark for the rest of this exercise.

Experiment 2 | Now, switch to *G-VARS* but keep the inputs uncorrelated and uniformly distributed - this module is essentially the same as the previous module but with a more generalized method, with the objective of testing if G-VARS can re-produce VARS results.

Experiment 3 | Run *G-VARS* when the inputs are correlated but still uniformly distributed - the objective of this module is to assess the role of correlation between inputs on the SA results.

Experiment 4 | Finally, run the previous experiment now with both correlated and non-uniformly distributed inputs - the objective of this module is to see what happens when the inputs follow other distributions such as normal or triangular distribution.

First import the needed librares, including `VARS` for VARS, `GVARS` for G-VARS and the `Model` class for creating a wrapper around the desired model.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import stats

from varstool import VARS, GVARS, Model

### Introduce the Ishigami function for sensitivity analysis

The Ishigami function is a function with three inputs. It is one of the most commonly used functions in the SA literature to test and compare the performance of different SA methods.

In [2]:
def ishigami(x, a=7, b=0.1):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

Wrap the function of interest with the `Model` class.

In [3]:
ishigami_model = Model(ishigami)

Let's run the wrapped function for an arbitrary input and check the output.

In [4]:
x=pd.Series({#name  #value
             'x1'   : 0 ,
             'x2'   : 0 ,
             'x3'   : 0 ,
             })
ishigami_model(x)

0.0

### Set up the four SA experiments

***
<p><center>Table 1. The attributes of the STAR-VARS algorithm </center></p>

| Attribute      | Description |
| :-------------:|:----------- |
|`paramaters`    | The name of every paramter along with its upper and lower bounds           |
|`num_stars`     | The total number of stars centers for VARS analysis                        |
|`delta_h`       | The sampling resolution of the STAR-VARS sampling to generate star points  |
|`ivars_scales`  | The scales of interest for IVARS estimation, e.g, 0.1 and 0.5 correspond (0-0.1) and (0-0.5) <br /> note: can not have a scale larger than 0.5|
|`star_centres`  | User-generated star centers - only used when a sampler is not chosen       |
|`sampler`       | The sampling strategy: `rnd`, `lhs`, `plhs`, `sobol_seq`, or `halton_seq` for generation of star centers|
|`slice_size`       | The slice size used when `plhs` sampling method is chosen|
|`seed`          | The seed number for randomization of the sampling strategy specified by `sampler`, <br /> only needed if a sampler was chosen  |
|`model`         | The wrapper of your model in the `Model` class|
|`bootstrap_flag`| This is a `True`/`False` value to turn on/off bootstrapping of VARS results   |
|`bootstrap_size`| The number of sampling iterations with replacement via bootstrapping |
|`bootstrap_ci`  | The level of confidence used in bootstrap reporting         |
|`grouping_flag` | This is a `True`/`False` value to turn on/off grouping of VARS results   |
|`num_grps`      | The number of groups you want to split your model paramaters into, <br /> if left blank the optimal number of groups will be calculated by VARS|
|`report_verbose`| this is a `True`/`False` value that if `True` will display a loading bar <br /> to show the progession of the VARS analysis, else there will be no progression loading bar|


***
<p><center>Table 2. The attributes that GVARS inherits from VARS </center></p>

| Attribute      | Description |
| :-------------:|:----------- |
|`num_stars`     | The total number of stars centers for VARS analysis                        |
|`ivars_scales`  | The scales of interest for IVARS estimation, e.g, 0.1 and 0.5 correspond (0-0.1) and (0-0.5) <br /> note: can not have a scale larger than 0.5|
|`star_centres`  | User-generated star centers - only used when a sampler is not chosen       |
|`sampler`       | The sampling strategy: `rnd`, `lhs`, `plhs`, `sobol_seq`, or `halton_seq` for generation of star centers|
|`seed`          | The seed number for randomization of the sampling strategy specified by `sampler`, <br /> only needed if a sampler was chosen  |
|`model`         | The wrapper of your model in the `Model` class|
|`bootstrap_flag`| This is a `True`/`False` value to turn on/off bootstrapping of VARS results   |
|`bootstrap_size`| The number of sampling iterations with replacement via bootstrapping |
|`bootstrap_ci`  | The level of confidence used in bootstrap reporting         |
|`grouping_flag` | This is a `True`/`False` value to turn on/off grouping of VARS results   |
|`num_grps`      | The number of groups you want to split your model paramaters into, <br /> if left blank the optimal number of groups will be calculated by VARS|
|`report_verbose`| this is a `True`/`False` value that if `True` will display a loading bar <br /> to show the progession of the VARS analysis, else there will be no progression loading bar|

***
In addition, GVARS has three unique attributes: `parameters`, `corr_mat`, and `num_dir_samples`, as explained in the following.

`parameters` includes the name of every model parameter, the distribution it follows, and the parameters specifying that distribution. The distributions currently available and their parameters are listed in Table 2.

<p><center>Table 2. The parameter distributions currently supported in GVARS </center></p>

| Distribution type        | Distribution parameter 1 | Distribution parameter 2 | Distribution parameter 3 | Distribution identifier |
| :-----------------------:|:-----------------------: |:-----------------------: |:-----------------------: |:----------: |
|uniform                   | lower bound              | upper bound              | None                     |  `unif`     |
|triangle                  | lower bound              | upper bound              | mode                     |  `triangle` |
|normal                    | mean                     | standard deviation       | None                     |  `norm`     |
|lognormal                 | mean                     | standard deviation       | None                     |  `lognorm`  |
|exponential               | mean                     | standard deviation       | None                     |  `expo`     |
|generalized extreme value | location                 | scale                    | shape                    |  `gev`      |


`corr_mat` is the correlation matrix which includes the Pearson correlation between every pair of parameters and must be defined as a numpy array.


`num_dir_samples` is the number of samples GVARS takes along each direction for every star center. 
***
**A "good-to-know" peice for advanced users:**

In GVARS, the star-based sampling is modified to work with `num_stars` and `num_dir_samples`. This is unlike VARS where the star-based sampling works with `num_stars` and `delta_h`.

GVARS still needs `delta_h` as an attribute for estimating and reporting directional variograms (not for sampling). As a rule-of-thumb, the user may set `delta_h` to be the reciprocal of `num_dir_samples`. For example, if num_dir_samples = 10, we suggest delta_h = 0.1.

An advanced user may try other values of `delta_h` to extract more accurate results. In general, the smaller the `delta_h`, the more granular the results. But, the tradeoff with smaller values would be to have fewer pairs in the bins when estimating variograms.


In [5]:
# Define Experiment 1

my_parameters = { 'x1': [ -3.14, 3.14 ], 
                  'x2': [ -3.14, 3.14 ], 
                  'x3': [ -3.14, 3.14 ], }

experiment_1 = VARS(parameters     = my_parameters,
                    num_stars      = 100,
                    delta_h        = 0.1,
                    ivars_scales   = (0.1, 0.3, 0.5),
                    sampler        = 'lhs',
                    seed           = 123456789,
                    model          = ishigami_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci   = 0.9,
                    grouping_flag  = True,
                    num_grps       = 2,
                    report_verbose = True,
                    )

In [6]:
# Define Experiment 2

my_parameters = { 'x1': (-3.14, 3.14, None, 'unif'), 
                  'x2': (-3.14, 3.14, None, 'unif'), 
                  'x3': (-3.14, 3.14, None, 'unif')}

my_corr_mat = np.array([[ 1   ,  0 ,  0   ],
                        [ 0   ,  1 ,  0   ],
                        [ 0   ,  0 ,  1   ]])

my_num_dir_samples = 10

my_delta_h = 0.1
# my_delta_h = 1 / my_num_dir_samples # or choose values such as 0.1

experiment_2 = GVARS(parameters     = my_parameters,
                    corr_mat        = my_corr_mat,
                    num_stars       = 100,
                    num_dir_samples = my_num_dir_samples,
                    delta_h         = my_delta_h,
                    ivars_scales    = (0.1, 0.3, 0.5),
                    sampler         = 'plhs',
                    slice_size      = 10,
                    model           = ishigami_model,
                    seed            = 123456789,
                    bootstrap_flag  = True,
                    bootstrap_size  = 100,
                    bootstrap_ci    = 0.9,
                    grouping_flag   = False,
                    num_grps        = 2,
                    report_verbose  = True,
                    )

In [7]:
# Define Experiment 3

my_parameters = { 'x1': (-3.14, 3.14, None, 'unif'), 
                  'x2': (-3.14, 3.14, None, 'unif'), 
                  'x3': (-3.14, 3.14, None, 'unif')}

my_corr_mat = np.array([[ 1   ,  0 ,  0.8 ],
                        [ 0   ,  1 ,  0   ],
                        [ 0.8 ,  0 ,  1   ]])

my_num_dir_samples = 10

my_delta_h = 0.1
# my_delta_h = 1 / my_num_dir_samples # or choose values such as 0.1

experiment_3 = GVARS(parameters     = my_parameters,
                    corr_mat        = my_corr_mat,
                    num_stars       = 100,
                    num_dir_samples = my_num_dir_samples,
                    delta_h         = my_delta_h,
                    ivars_scales    = (0.1, 0.3, 0.5),
                    sampler         = 'plhs',
                    slice_size      = 10,
                    model           = ishigami_model,
                    seed            = 123456789,
                    bootstrap_flag  = True,
                    bootstrap_size  = 100,
                    bootstrap_ci    = 0.9,
                    grouping_flag   = True,
                    num_grps        = 2,
                    report_verbose  = True,
                    )

In [8]:
# Define Experiment 4

my_parameters = { 'x1': (-3.14, 3.14,  None, 'unif'    ), 
                  'x2': (  0  ,  1  ,  None, 'norm'    ), 
                  'x3': (-3.14, 3.14, -3.14, 'triangle')}

my_corr_mat = np.array([[ 1   ,  0 ,  0.8 ],
                        [ 0   ,  1 ,  0   ],
                        [ 0.8 ,  0 ,  1   ]])

my_num_dir_samples = 10

my_delta_h = 0.1
# my_delta_h = 1 / my_num_dir_samples # or choose values such as 0.1

experiment_4 = GVARS(parameters     = my_parameters,
                    corr_mat        = my_corr_mat,
                    num_stars       = 100,
                    num_dir_samples = my_num_dir_samples,
                    delta_h         = my_delta_h,
                    ivars_scales    = (0.1, 0.3, 0.5),
                    sampler         = 'plhs',
                    slice_size      = 10,
                    model           = ishigami_model,
                    seed            = 123456789,
                    bootstrap_flag  = True,
                    bootstrap_size  = 100,
                    bootstrap_ci    = 0.9,
                    grouping_flag   = True,
                    num_grps        = 2,
                    report_verbose  = True,
                    )

### Run the SA experiments

Now, run the four experiments defined above.

In [9]:
experiment_1.run_online()

Star Centres: 100 Centers Loaded
Star Points: Not Loaded
Parameters: 3 paremeters set
Delta h: 0.1
Model: ishigami
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
Verbose: On
VARS Analysis: Not Done

In [10]:
experiment_2.run_online()

building fictive matrix:   0%|                                                                   | 0/2 [00:00<…

generating star points
:   0%|                                                                   | 0/8 [00:00<…

model evaluation:   0%|                                                                       | 0/3000 [00:00<…

building pairs:   0%|                                                                          | 0/300 [00:00<…

calculating 'h' values:   0%|          | 0/100 [00:00<?, ?it/s]

binning and reording pairs based on 'h' values:   0%|          | 0/100 [00:00<?, ?it/s]

VARS analysis:   0%|                                                                            | 0/10 [00:00<…

factor ranking:   0%|                                                                            | 0/2 [00:00<…

bootstrapping:   0%|                                                                           | 0/100 [00:00<…

In [11]:
experiment_3.run_online()

building fictive matrix:   0%|                                                                   | 0/2 [00:00<…

generating star points
:   0%|                                                                   | 0/8 [00:00<…

model evaluation:   0%|                                                                       | 0/3000 [00:00<…

building pairs:   0%|                                                                          | 0/300 [00:00<…

calculating 'h' values:   0%|          | 0/100 [00:00<?, ?it/s]

binning and reording pairs based on 'h' values:   0%|          | 0/100 [00:00<?, ?it/s]

VARS analysis:   0%|                                                                            | 0/10 [00:00<…

factor ranking:   0%|                                                                            | 0/2 [00:00<…

bootstrapping and grouping:   0%|                                                              | 0/100 [00:00<…

In [12]:
experiment_4.run_online()

building fictive matrix:   0%|                                                                   | 0/2 [00:00<…

generating star points
:   0%|                                                                   | 0/8 [00:00<…

model evaluation:   0%|                                                                       | 0/3000 [00:00<…

building pairs:   0%|                                                                          | 0/300 [00:00<…

calculating 'h' values:   0%|          | 0/100 [00:00<?, ?it/s]

binning and reording pairs based on 'h' values:   0%|          | 0/100 [00:00<?, ?it/s]

VARS analysis:   0%|                                                                            | 0/10 [00:00<…

C:\Programs\Anaconda3\lib\site-packages\varstool\varstool.py:1455: UserWarning: For some or all variables there are no results for the directional variogram for the following h values: 0.8, you may want to consider increasing the sample size.

  warnings.warn(warning_string,


factor ranking:   0%|                                                                            | 0/2 [00:00<…

bootstrapping and grouping:   0%|                                                              | 0/100 [00:00<…

### Check out the results

Let's check out the results of the four SA experiments, including IVARS, VARS-TO, VARS-ABE, VARS-ACE, and directional variograms.

**IVARS: Integrated variogram Across a Range of Scales** 

IVARS indices are the primary sensitivity indices by the VARS approach. First, print all the IVARS indices for the scale ranges of interest.

In [13]:
# IVARS from Experiment 1
cols = experiment_1.parameters.keys()
experiment_1.ivars[cols]

AttributeError: 'VARS' object has no attribute 'ivars'

In [ ]:
# IVARS from Experiment 2
cols = experiment_2.parameters.keys()
experiment_2.ivars[cols]

In [ ]:
# IVARS from Experiment 3
cols = experiment_3.parameters.keys()
experiment_3.ivars[cols]

In [ ]:
# IVARS from Experiment 4
cols = experiment_4.parameters.keys()
experiment_4.ivars[cols]

 Choose a scale range and plot the respective IVARS indices. Two points:
 
    POINT1: IVARS-50 (h=[0-0.5]), called ***Total-Variogram Effect*** is the most comprehensive sensitivity index.
    POINT2: Plotting sensitivity results in log scale helps us better differentiate less influential parameters.

In [ ]:
# Plot IVARS from Experiment 1
ivars_scale = 0.5 # Choose the scale range of interest, e.g., 0.1, 0.3, or 0.5

cols = experiment_1.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')


In [ ]:
# Plot IVARS from Experiment 2
ivars_scale = 0.5 # Choose the scale range of interest, e.g., 0.1, 0.3, or 0.5

cols = experiment_2.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_2.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_2.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

Experiment 2 shows similar results to experiment 1 in terms of senstivity for each of the parameters as expected

In [ ]:
# Plot IVARS from Experiment 3
ivars_scale = 0.5 # Choose the scale range of interest, e.g., 0.1, 0.3, or 0.5

cols = experiment_3.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_3.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_3.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

The noticable difference in experiment 3 comes from the new correlation between `x1` and `x3`. This is shown in the plots as `x1` has drastically changed in terms of senstivity.

In [ ]:
# Plot IVARS from Experiment 4
ivars_scale = 0.5 # Choose the scale range of interest, e.g., 0.1, 0.3, or 0.5

cols = experiment_4.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_4.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_4.ivars.loc[pd.IndexSlice[ ivars_scale ]][cols], color='gold')
plt.gca().set_title (r'Integrated variogram Across a Range of Scales $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'IVARS-50 (Total-Variogram Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

Experiment 4 shows different results than the rest of the experiments due to the addition of both correlated parameters `x1` and `x2` along with there being non-uniformly distributed inputs.

**VARS-TO: Sobol Total-Order Effect** 

In addition to the IVARS indices, VARS and GVARS estimate Sobol variance-based total-order effects as side products. Below, print them and plot them.

In [ ]:
# VARS-TO from Experiment 1
cols = experiment_1.parameters.keys()
experiment_1.st.to_frame().T[cols]

In [ ]:
# VARS-TO from Experiment 2
cols = experiment_2.parameters.keys()
experiment_2.st.to_frame().T[cols]

In [ ]:
# VARS-TO from Experiment 3
cols = experiment_3.parameters.keys()
experiment_3.st.to_frame().T[cols]

In [ ]:
# VARS-TO from Experiment 4
cols = experiment_4.parameters.keys()
experiment_4.st.to_frame().T[cols]

In [ ]:
# Plot VARS-TO from Experiment 1
cols = experiment_1.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

In [ ]:
# Plot VARS-TO from Experiment 2
cols = experiment_2.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_2.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_2.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

In [ ]:
# Plot VARS-TO from Experiment 3
cols = experiment_3.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_3.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_3.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

In [ ]:
# Plot VARS-TO from Experiment 4
cols = experiment_4.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_4.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_4.st.to_frame().T.iloc[0][cols], color='lightblue')
plt.gca().set_title (r'Sobol Total-Order Effect $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'VARS-TO (Total-Order Effect)', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('log')

The Sobol Total-Order Effect approach confirms our analysis from the IVARS-50 approach as the results are similar in shape.

**VARS-ABE & VARS-ACE: Morris Elementary Effects** 

In addition to the IVARS and Sobol indices, VARS and GVARS estimate various versions of Morris derivative-based elementary effects as side products, **mean ABsolute Elementary effect (ABE)** and **mean ACtual elementary effect (ACE)**.

    POINT: In the derivative-based approach, the user needs to choose a delta (step size) for numerical estimation of derivatives. Recommended is to go with the smallest delta available here, which is equal to delta_h, but the user could choose any integer product of delta_h as well.

 Below, print VARS-ABE & VARS-ACE and then plot them.

In [ ]:
# delta_of_interest = experiment_1.maee.to_frame().unstack(level=0).index.min() # This lines chooses the smallest delta
delta_of_interest = 0.1  # This line can be used to set other values for delta

In [ ]:
# VARS-ABE from Experiment 1
experiment_1.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ACE from Experiment 1
experiment_1.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ABE from Experiment 2
experiment_2.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ACE from Experiment 2
experiment_2.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ABE from Experiment 3
experiment_3.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ACE from Experiment 3
experiment_3.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ABE from Experiment 4
experiment_4.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# VARS-ACE from Experiment 4
experiment_4.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T

In [ ]:
# Plot VARS-ACE and  VARS-ABE from Experiment 1
cols = experiment_1.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='green')
plt.gca().set_title (r'Mean Absolute Elementary Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ABE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_1.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='lightgreen')
plt.gca().set_title (r'Mean Actual Elementary Effect ', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ACE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

In [ ]:
# Plot VARS-ACE and  VARS-ABE from Experiment 2
cols = experiment_2.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_2.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='green')
plt.gca().set_title (r'Mean Absolute Elementary Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ABE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_2.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='lightgreen')
plt.gca().set_title (r'Mean Actual Elementary Effect ', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ACE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

In [ ]:
# Plot VARS-ACE and  VARS-ABE from Experiment 3
cols = experiment_3.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_3.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='green')
plt.gca().set_title (r'Mean Absolute Elementary Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ABE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_3.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='lightgreen')
plt.gca().set_title (r'Mean Actual Elementary Effect ', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ACE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

In [ ]:
# Plot VARS-ACE and  VARS-ABE from Experiment 4
cols = experiment_4.parameters.keys()                     
fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_4.maee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='green')
plt.gca().set_title (r'Mean Absolute Elementary Effect', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ABE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

fig_bar = plt.figure(figsize=(10,5))
plt.gca().bar(cols, experiment_4.mee.to_frame().unstack(level=0).loc[delta_of_interest].to_frame().T.iloc[0], color='lightgreen')
plt.gca().set_title (r'Mean Actual Elementary Effect ', fontsize = 15)
plt.gca().set_ylabel(r'VARS-ACE', fontsize = 13)
plt.gca().set_xlabel(r'Model Parameter', fontsize=13)
plt.gca().grid()
plt.gca().set_yscale('linear')

As you may notice the derivate based approaches give us different results as the previous approaches (IVARS-50 and Sobol Total-Order Effect) were based on variograms and variance which are related to eachother. This gives us another perspective on the determination of different variable sensitivities.

**Directional Variograms - γ(h)** 

For advanced users of VARS, directional variograms may provide a wealth of information about the structure of the model response surface.

    POINT: Variograms are most meaningful when h (perturbation scale) is between zero and 0.5 of the parameter ranges, but the user can investigate the entire perturbation range (0 – 1).
    
 Below, print the directional variograms and then plot them.

In [ ]:
# Directional Variograms from Experiment 1
cols = experiment_1.parameters.keys()
variograms1 = experiment_1.gamma.unstack(0)[cols].copy()
variograms1

In [ ]:
# Directional Variograms from Experiment 2
cols = experiment_2.parameters.keys()                     
variograms2 = experiment_2.gamma.unstack(0)[cols].copy()
variograms2

In [ ]:
# Directional Variograms from Experiment 3
cols = experiment_3.parameters.keys()                     
variograms3 = experiment_3.gamma.unstack(0)[cols].copy()
variograms3

In [ ]:
# Directional Variograms from Experiment 4
cols = experiment_4.parameters.keys()                     
variograms4 = experiment_4.gamma.unstack(0)[cols].copy()
variograms4

In [ ]:
# Plot Directional Variograms from Experiment 1

plotting_scale = 0.5 # any number between delta_h and one.

matrix_y = variograms1.loc[variograms1.index <= plotting_scale].to_numpy()
column_x = variograms1.loc[variograms1.index <= plotting_scale].index.to_numpy()
matrix_x = np.tile(column_x, (matrix_y.shape[1], 1)).T

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('linear')
plt.gca().legend (cols, loc='upper left', fontsize = 10)
plt.gca().grid()

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('log')
plt.gca().legend (cols, loc='lower right', fontsize = 10)
plt.gca().grid()

In [ ]:
# Plot Directional Variograms from Experiment 2

plotting_scale = 0.5 # any number between delta_h and one.

matrix_y = variograms2.loc[variograms2.index <= plotting_scale].to_numpy()
column_x = variograms2.loc[variograms2.index <= plotting_scale].index.to_numpy()
matrix_x = np.tile(column_x, (matrix_y.shape[1], 1)).T

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('linear')
plt.gca().legend (cols, loc='upper left', fontsize = 10)
plt.gca().grid()

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('log')
plt.gca().legend (cols, loc='lower right', fontsize = 10)
plt.gca().grid()

In [ ]:
# Plot Directional Variograms from Experiment 3

plotting_scale = 0.5 # any number between delta_h and one.

matrix_y = variograms3.loc[variograms3.index <= plotting_scale].to_numpy()
column_x = variograms3.loc[variograms3.index <= plotting_scale].index.to_numpy()
matrix_x = np.tile(column_x, (matrix_y.shape[1], 1)).T

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('linear')
plt.gca().legend (cols, loc='upper left', fontsize = 10)
plt.gca().grid()

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('log')
plt.gca().legend (cols, loc='lower right', fontsize = 10)
plt.gca().grid()

In [ ]:
# Plot Directional Variograms from Experiment 4

plotting_scale = 0.5 # any number between delta_h and one.

matrix_y = variograms4.loc[variograms4.index <= plotting_scale].to_numpy()
column_x = variograms4.loc[variograms4.index <= plotting_scale].index.to_numpy()
matrix_x = np.tile(column_x, (matrix_y.shape[1], 1)).T

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('linear')
plt.gca().legend (cols, loc='upper left', fontsize = 10)
plt.gca().grid()

fig_cdf = plt.figure(figsize=(10,5))
plt.gca().plot(matrix_x, matrix_y )
plt.gca().set_title (r'Directional Variogram $[log-scale]$', fontsize = 15)
plt.gca().set_ylabel(r'$γ(h)$', fontsize = 13)
plt.gca().set_xlabel(r'$h$ (perturbation scale)', fontsize=13)
plt.gca().set_yscale('log')
plt.gca().legend (cols, loc='lower right', fontsize = 10)
plt.gca().grid()

**Bootstrap results for assessing the uncertainty and robustness of SA** 

The user of SA may need to assess the uncertainty around any of the results reported above and their robustness against sampling size and variability. To do so, when setting up the SA experiment above, make sure to turn on the bootstrap_flag, and choose a bootrtap size and confidence interval, like shown bellow.

bootstrap_flag = True,

bootstrap_size = 1000, (recommended to be greater than 100; the bigger the number the more accurate the estimate)

bootstrap_ci   = 0.9, (a confidence interval of 0.9 means that with a likelihood of 90%, the true value is within the intervals generated by bootstrap)

Bootstrap-generated upper and lower bounds:

In [ ]:
# IVARS from Experiment 1 with bootstrap
cols = experiment_1.parameters.keys()
print('Upper Bounds on IVARS based on bootstrapping'); display(experiment_1.ivarsub[cols])
print('Lower Bounds on IVARS based on bootstrapping'); display(experiment_1.ivarslb[cols])

In [ ]:
# Plot IVARS50, VARS-TO, and VARS-ABE along with their confidence intervals from Experiment 1
ax =experiment_1.factorimportance_plot()

Factor rankings and their bootstrap-generated robustness:
 
     () Rank zero indicates the most influential parameter, one the second most influential, etc.
     
     () The higher the robustness value (also called reliability) the better (maximum one). A robustness of one for a parameter means that parameter received its respective rank in every instance of bootstrap re-samplings (maximum robustness). And, for example, a robustness of 0.7 means the respective rank turned out to be the case in 70% of bootstrap re-samplings.



In [ ]:
# Rankings and Robustness from Experiment 1
cols = experiment_1.parameters.keys()
print('Parameter Rankings'); display(experiment_1.ivars_factor_ranking[cols])
print('Robustness of Rankings'); display(experiment_1.rel_ivars_factor_ranking[cols])

In the case of high-dimensional problems (models with many inputs), the user may want to group parameters together based on their importance characterized by SA. To do so, when setting up the SA experiment above, make sure to turn on the grouping_flag and choose the number of groups of interest. Also, make sure that the bootstrap_flag is on.

grouping_flag  = True, 
num_grps       = 2, (any number smaller than the number of parameters)

In [ ]:
# Grouping of parameters from Experiment 1
cols = experiment_1.parameters.keys()
experiment_1.ivars50_grp[cols]

### Delve deeper - Additional information for advanced users

The information below are also available to the user, if there is interest in further exploring what is happening under the hood.

**Cross sections of sample points in the input space**

Choose any pair of parameters and see the projection of all star centers and star points onto their two-dimensional space.

In [ ]:
names = ['x1', 'x2']; experiment_1.correlation_plot(param_names=names)
names = ['x1', 'x3']; experiment_1.correlation_plot(param_names=names)
names = ['x2', 'x3']; experiment_1.correlation_plot(param_names=names)

In [ ]:
experiment_2.correlation_plot(param_names=['x1', 'x2'])
experiment_2.correlation_plot(param_names=['x1', 'x3'])
experiment_2.correlation_plot(param_names=['x2', 'x3'])

In [ ]:
experiment_3.correlation_plot(param_names=['x1', 'x2'])
experiment_3.correlation_plot(param_names=['x1', 'x3'])
experiment_3.correlation_plot(param_names=['x2', 'x3'])

In [ ]:
experiment_4.correlation_plot(param_names=['x1', 'x2'])
experiment_4.correlation_plot(param_names=['x1', 'x3'])
experiment_4.correlation_plot(param_names=['x2', 'x3'])

**Star centers used in the GVARS experiment**

The following dataframe includes all the star centers generated.

In [ ]:
# Star centers generated in Experiment 1
cols = experiment_1.parameters.keys()
pd.DataFrame(data=experiment_1.star_centres, columns=cols)

In [ ]:
# Star centers generated in Experiment 2
cols = experiment_2.parameters.keys()
pd.DataFrame(data=experiment_2.star_centres, columns=cols)

In [ ]:
# Star centers generated in Experiment 3
cols = experiment_3.parameters.keys()
pd.DataFrame(data=experiment_3.star_centres, columns=cols)

In [ ]:
# Star centers generated in Experiment 4
cols = experiment_4.parameters.keys()
pd.DataFrame(data=experiment_4.star_centres, columns=cols)

**Star points generated along with respective outputs**

The following dataframe includes all the star points generated. For each row, the first column indicates the associated star center, the second column indicates the parameter along which the cross-sectional samples were taken, and the third column indicates the star point numbers along that cross section. The last column indicates the evaluated model response (output) for the input set in each row.

In [ ]:
# Star points generated in Experiment 1
experiment_1.model_df

In [ ]:
# Star points generated in Experiment 2
experiment_2.model_df

In [ ]:
# Star points generated in Experiment 3
experiment_3.model_df

In [ ]:
# Star points generated in Experiment 4
experiment_4.model_df

**Pairs generated and their associated dissimilarity**

The following dataframe includes all the resulting pairs of samples. For each row, the first column indicates the associated star center and the second column indicates the parameter along which the cross-sectional samples were taken.
The 'pair_ind' column indicates the ID of sample points in each pair, the 'h' column indicates the binned distance between the two samples in each pair, and the 'actual h' column indicates the exact distance between the two. The columns '0' and '1' shows the outputs [y1 and y2] associated with each sample points in the pair. The 'dissimilarity' column shows 0.5[y1-y2]^2.

In [ ]:
# all pairs generated in Experiment 1
experiment_1.pair_df

In [ ]:
# all pairs generated in Experiment 2
experiment_2.pair_df

In [ ]:
# all pairs generated in Experiment 3
experiment_3.pair_df

In [ ]:
# all pairs generated in Experiment 4
experiment_4.pair_df

**The fictive correlation matrix**

The fictive correlation matrix is a transformation of the original correlation matrix to a normal standard space.

In [ ]:
fictive_matrix = experiment_2.cov_mat; cols = experiment_2.parameters.keys()
pd.DataFrame(data = fictive_matrix, index = cols, columns = cols).round(decimals=2)

In [ ]:
fictive_matrix = experiment_3.cov_mat; cols = experiment_3.parameters.keys()
pd.DataFrame(data = fictive_matrix, index = cols, columns = cols).round(decimals=3)

In [ ]:
fictive_matrix = experiment_4.cov_mat; cols = experiment_4.parameters.keys()
pd.DataFrame(data = fictive_matrix, index = cols, columns = cols).round(decimals=3)

All done!